In [39]:
# モデルを読み込む
from sentence_transformers import SentenceTransformer

model_name = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
model = SentenceTransformer(model_name)


In [40]:
import csv

# CSVファイルを読み込む
# https://w1740803485-clv347541.slack.com/archives/C08F7JZPD63/p1745221416748989?thread_ts=1745208882.923669&cid=C08F7JZPD63
# https://github.com/digitaldemocracy2030/kouchou-ai/issues/280
source_table = []
with open('eng_pubcom.csv', 'r', encoding="utf-8") as file:
    reader = csv.reader(file)
    for row in reader:
        source_table.append(row)

header = source_table[0]
source_table = source_table[1:]

# データ構造はid、カテゴリ番号、要約、本文の順
source_table[:10]


[['1', '1', 'エネルギー基本計画に賛成である', '当社は地球温暖化防止の方向性について賛同し、第7次エネルギー基本計画に賛同いたします。'],
 ['2',
  '1',
  'エネルギー基本計画に賛成である',
  '「バランスのとれたＳ＋３Ｅの実現」を基本方針として検討、策定されている点について、賛同する。'],
 ['3',
  '1',
  'エネルギー基本計画に賛成である',
  '今回示されたエネルギー基本計画案は、S＋3Eの原則のもと、わが国の国民生活、経済活動を支えるエネルギー政策で、資源に乏しい島国であるわが国の地政学的なリスクも現実的に捉えており、脱炭素と経済性・安定供給のバランスを目指した内容で、概ね評価できるものである。とくに製紙業界では製造工程で大量の蒸気エネルギーが不可欠であり、一足飛びで火力の燃料を全てカーボンニュートラルとはならない状況を踏まえたエネルギーミックスとなっていることに賛同する。'],
 ['4', '1', 'エネルギー基本計画に賛成である', '第７次エネルギー基本計画（案）の方向性に賛同します'],
 ['5',
  '1',
  'エネルギー基本計画に賛成である',
  '国の示す「我が国が将来にわたって豊かな国として存続し、全ての国民が希望をもって暮らせる社会を実現するためには、エネルギー安定供給、経済成長、脱炭素を同時に実現していく必要がある」という認識ならびに目指す方向性に賛同する。'],
 ['6',
  '1',
  'エネルギー基本計画に賛成である',
  '第7次エネルギー基本計画（案）では、ウクライナ侵攻や中東情勢の緊迫化等に端を発したエネルギー安全保障の要請の高まり、DX,GX等の進展に伴う電力需要増加の見通し等エネルギーを取り巻く情勢変化を正面から捉え、S+3Eの観点は堅持しつつも、第6次エネルギー基本計画において「環境適合」に過度に傾斜していた部分を、「安定供給」、「経済効率性」の重要性を改めて捉え直した現実的な計画とされたことは、産業界の危機感と軌を一にするものと評価する。'],
 ['7',
  '1',
  'エネルギー基本計画に賛成である',
  '今般示された第7次エネルギー基本計画、地球温暖化対策計画、GX2040ビジョンは、エネルギー政策と、その裏腹の関係にある地球

In [42]:
# source_table は id, カテゴリ番号、要約, 本文の構造
# idと本文 だけを抽出する、重複を排除する

already_seen = set()
already_seen.add("") # 空白だけのコメントを除去するために、予め追加しておく

target_table = []
for row in source_table:
    id = row[0]
    data = " ".join(row[3:]) # カンマで繋がっている可能性があるので、joinしておく
    if data not in already_seen:
        already_seen.add(data)
        target_table.append((id, data))
target_table[:3]

[('1', '当社は地球温暖化防止の方向性について賛同し、第7次エネルギー基本計画に賛同いたします。'),
 ('2', '「バランスのとれたＳ＋３Ｅの実現」を基本方針として検討、策定されている点について、賛同する。'),
 ('3',
  '今回示されたエネルギー基本計画案は、S＋3Eの原則のもと、わが国の国民生活、経済活動を支えるエネルギー政策で、資源に乏しい島国であるわが国の地政学的なリスクも現実的に捉えており、脱炭素と経済性・安定供給のバランスを目指した内容で、概ね評価できるものである。とくに製紙業界では製造工程で大量の蒸気エネルギーが不可欠であり、一足飛びで火力の燃料を全てカーボンニュートラルとはならない状況を踏まえたエネルギーミックスとなっていることに賛同する。')]

In [43]:
# エンベデッドする
sentences = [row[1] for row in target_table]
embeddings = model.encode(sentences)
print(embeddings.shape)

(1006, 768)


In [44]:
# 埋め込みベクトルから、コサイン類似度行列を算出する
similarities = model.similarity(embeddings, embeddings)
print(similarities)

tensor([[1.0000, 0.5705, 0.6571,  ..., 0.5793, 0.4519, 0.7250],
        [0.5705, 1.0000, 0.7210,  ..., 0.3734, 0.3809, 0.4115],
        [0.6571, 0.7210, 1.0000,  ..., 0.5277, 0.6193, 0.5714],
        ...,
        [0.5793, 0.3734, 0.5277,  ..., 1.0000, 0.4356, 0.7028],
        [0.4519, 0.3809, 0.6193,  ..., 0.4356, 1.0000, 0.3931],
        [0.7250, 0.4115, 0.5714,  ..., 0.7028, 0.3931, 1.0000]])


In [48]:
# 類似度が高いものを抽出
import numpy as np

# score, text1, text2 の順で格納
similar_texts = []

# 類似度が0.8以上のインデックスを取得
high_similarity_indices = np.where(0.8 < similarities)

# 高い類似度のペアを抽出
for idx in range(len(high_similarity_indices[0])):    
    i, j = high_similarity_indices[0][idx], high_similarity_indices[1][idx]
    if j >= i:
        continue

    score = float(similarities[i, j])
    similar_texts.append((score, target_table[i][0], sentences[i], target_table[j][0], sentences[j]))

# 類似度でソート
similar_texts.sort(key=lambda x: x[0], reverse=True)
similar_texts[:30]

[(1.0000001192092896,
  '1095',
  '●該当箇所(P65-)８．エネルギーシステム改革（２）脱炭素と安定供給を実現する持続的な電力システムの構築へ向けた取組② 電力システムが直面する課題と対応方針(イ) 電源の効率的活用に向けた系統整備・立地誘導と柔軟な需給運用の仕組み構築（前略）併せて、再生可能エネルギーが大量に導入される中で、需給運用を高度化していくため、需給調整市場について、更なる運用改善を進めていき、中長期的には、系統制約も考慮した上で、「同時市場」の導入に向けて、本格的に検討を深めていく。●意見内容・また、現在議論されている「同時市場」については、具体的かつ実現性のある市場となるよう、適宜情報を公表するとともに、様々な関係者の意見を聴取いただくようお願いします。●理由（可能であれば、根拠となる出典等を添付又は併記して下さい。）・また、現在検討されている「同時市場」については、需給運用の高度化と系統制約を考慮したものとされていますが、緒に就いたばかりの各市場がどのように統合整理されるのかなど、明確になっていないとの認識です。・こうしたことから、事業者の中長期的な予見性確保のためにも、関係者との情報連携が必要との認識です。',
  '1091',
  '●該当箇所(P65-)８．エネルギーシステム改革（２）脱炭素と安定供給を実現する持続的な電力システムの構築へ向けた取組② 電力システムが直面する課題と対応方針(イ) 電源の効率的活用に向けた系統整備・立地誘導と柔軟な需給運用の仕組み構築（前略）併せて、再生可能エネルギーが大量に導入される中で、需給運用を高度化していくため、需給調整市場について、更なる運用改善を進めていき、中長期的には、系統制約も考慮した上で、「同時市場」の導入に向けて、本格的に検討を深めていく。●意見内容・需給調整市場の更なる運用改善に当たっては、市場メカニズムに留意しつつ、創意工夫された新規リソースが参入しやすいルール等の構築をお願いします。●理由（可能であれば、根拠となる出典等を添付又は併記して下さい。）・需給調整市場は、今年度全商品の取扱いがスタートしましたが、応札量不足や度重なる募集ルール変更により、市場として機能しているとは言い難い面があります。・市場機能を活性化させるためには、新規参入が不可欠であるこ

In [ ]:
# similar_textsをCSVに書き出す
import pandas as pd
similar_texts_df = pd.DataFrame(similar_texts, columns=['score', 'id1', 'text1', 'id2', 'text2'])
similar_texts_df.to_csv('similar_texts_energy.csv', index=False, encoding='utf-8-sig')
similar_texts_df.to_excel('similar_texts_energy.xlsx', index=False, encoding='utf-8-sig')